# Numerical solving system of linear equations and initial problem for ODE

## Numerical solving system of linear equations

In [4]:
import numpy as np
import matplotlib.pyplot as plt

### 1. Introduction and motivation

One of the basic problems in numerical mathematics is solving system of linear equations. In this essay we will attempt to explore methods for solving quadratic $n\times n$ systems, or system of $n$ equations with $n$ unknowns,

$$
\begin{equation}
a_{11}x_{1} + a_{12}x_{2} + ... + a_{1j}x_{j} + ... + a_{1n}x_{n} = b_{1}\\
a_{21}x_{1} + a_{22}x_{2} + ... + a_{2j}x_{j} + ... + a_{2n}x_{n} = b_{2}\\
\vdots\\
a_{i1}x_{1} + a_{i2}x_{2} + ... + a_{ij}x_{j} + ... + a_{in}x_{n} = b_{i}\\
\vdots\\
a_{n1}x_{1} + a_{n2}x_{2} + ... + a_{nj}x_{j} + ... + a_{nn}x_{n} = b_{n}\\
\end{equation}
$$

We will denote matrix as $A=(a_{ij})_{i,j=1}^{n}$, and refer to it as <b>system matrix</b>, its elements are called <b>coefficients of the system</b>. Vector $b=(b_{i})_{i=1}^{n}$ is called <b>right-hand side</b> vector (RHS in short). We need to solve for $x$, where $x$ is <b>vector of unknowns</b> $x=(x_{i})_{i=1}^{n}$ such that $Ax=b$ holds.

In theory of linear algebra, solving $Ax=b$ is a almost trivial problem, especially when system matrix is quadratic and regular. Solution $x$ is given by equation $x=A^{-1}b$, where $A^{-1}$ is the inverse matrix for $A$ ($AA^{-1}=A^{-1}A=I$). There are explicit formulae for elements of matrix $A^{-1}$ and for the solution $x$. Besides, the well-known Gaussian elimination solves for $x$ in $O(n^{3})$ elementary operations (where elementary operation is addition, subtraction, multiplication, or division).

We know that the solution $x=A^{-1}b$ exists, it is unique, and there is a simple algorithm that explicitly solves for $x$ using a finite number of simple arithmetic operations.
Sadly, the finite arithmetic of a computer cannot even execute these simple operations exactly, so using Gaussian eliminations (which are simply a finite sequence of formulae which lead to the solution) in general we cannot compute the solution of linear system $Ax=b$ absolutely correct.

For that reason we introduce iterative methods. Iterative methods do not give us a solution, but the approximation of a solution. As it will be shown later, using iterative methods we do not know the exact solution, but we know how close we are to the exact solution.

The methods we will focus on are the Jacobi method, the Gauss-Seidel method, and the method of successive over-relaxation (SOR method, in short).

### 2. Base of methods

If we assume $Ax=b$, $det(A)\neq0$, and we keep in mind that instead of exact solution $x$ we are solving for some approximation $\tilde{x}$, we have to have a way to evaluate the quality of the approximation. One way of doing so is, obviously, to calculate the norm of $\delta x$, where $\delta x = \tilde{x} - x$, but that is not possible for $x$ is unknown. The other way is to calculate the residual

$$ r = b - A\tilde{x} \label{eq:1}\tag{1} $$
Let's say that $\tilde{x}$ as a good approximation if, in some norm $||\cdot||$ $\epsilon:=\frac{||r||}{||b||}$ is small enough.

We can justify such criterion by saying that using $\eqref{eq:1}$ we get
$$ A\tilde{x} = \tilde{b}\equiv b-r, \frac{||\tilde{b}-b||}{||b||}=\frac{||r||}{||b||}=\epsilon.$$

We say $\tilde{x}$ is an exact solution to the system that is close to the default, with a controlled difference in right-hand side vector $b$.

This discussion motivates us to look for a different approach to solving system of linear equations $Ax=b$. Notice that we  do not have to strive to finding the exact solution. So, we aim for <i>good enough</i> approximation $\tilde{x}\approx A^{-1}b$. That is why it makes sense to try and construct a sequence $x^{(0)},x^{(1)},x^{(2)},...,x^{(k)},...$ of vectors with following properties:
- For each $k\in\mathbb{N}$ the formula to compute $x^{(k)}$ is a simple one and matrix $A$ is used only as a subprogram that computes $v\mapsto f(A)v$, where $v$ is a vector, and $f(A)$ denotes $A,A^{*},A^{T}$ or some part of $A$ (eg. $diag(A)$, upper or lower triangular matrix of $A$, etc.)
- $x^{(k)}$ converges to $x=A^{-1}b$ and for some $k$ (usually $k\ll n$) $x^{(k)}$ is an acceptable approximation for $x$.

These properties are given in such imprecise form on purpose. Details which depend on specific probblem and specific construction of a sequence $(x^{(k)}$ will be given when describing a specific method

Let's write matrix $A$ as $A=M-N$, where $M$ is a regular matrix and $M^{-1}A\approx I$. We have:

$$Mx=Nx+b,\;\text{ or }\; x=M^{-1}Nx+M^{-1}b \label{eq:2}\tag{2}$$

Let's denote $F:=M^{-1}N$ and $c:=M^{-1}b$. Now we have a fixed point problem $x=Fx+c$, with $F=M^{-1}N=M^{-1}(M-A)=I-M^{-1}A$ so it is only natural to try simple iterations in form

$$ x^{(k+1)}=Fx^{(k)}+c. \label{eq:3}\tag{3} $$

Inserting $F=I-M^{-1}A$ and $c=M^{-1}b$ we can see that iterations in $\eqref{eq:3}$ can be written as

$$ x^{(k+1)}=x^{(k)}+M^{-1}r_{k},\;\text{where}\;\; r_{k}=Ax^{(k)} \label{eq:4}\tag{4}$$

Intuitively, if $M$ is chosen in such a way that we know how to effectively use $M^{-1}$ and such that $M^{-1}\approx A^{-1}$, then 

$$ x^{(k+1)}=x^{(k)} + M^{-1}r_{k}\approx x^{(k)} + A^{-1}r_{k}= x^{(k)} + x - x^{(k)}=x.$$

We say that $M$ is a preconditioner for $A$ in a sense that $M^{-1}$ approximates $A^{-1}$. If we are given a good matrix $M$, then the system $Ax=b$ is equivalent to system $(M^{-1}A)x=M^{-1}b$ whose system matrix $M^{-1}A$ has better properties than $A$.

The biggest issue is how to decompose matrix $A$ as $A=M-N$ such that this decomposition will ensure convergence for certain problem class. For the methods we will be covering this decomposition is derived from this representation of matrix $A$:

$$ A = D(I-L-U) \label{eq:5}\tag{5}$$

$D=diag(A)$, $L$ is strictly lower triangular matrix, $U$ is strictly upper triangular matrix. The following representation will also be in use:
$$ A = D - \hat{L} - \hat{U},\; \hat{L}=DL,\; \hat{U}=DU$$

### Methods

#### Jacobi method

Jacobi method is defined for matrix $A\in\mathbb{M}_{n}$ for which $a_{ii}\neq0, \forall i=1,...,n$. Let matrix $M$ be the diagonal of $A$, $M=diag(A)$, which makes $N$ non-diagonal part of $-A$. We will denote matrix $F=D^{-1}(D-A)$ as $J$, where in terms of $\eqref{eq:5}$:

$$ J=L+U \label{eq:6}\tag{6}$$



Jacobi iterations $x^{(k+1)}=Jx^{(k)}+D^{-1}b$ on elements:

$$ x_i^{(k+1)}=\frac{1}{a_{ii}}(b_{i}-\sum_{j=1\\j\neq i}^{n}a_{ij}x_{j}^{(k)})\label{eq:7}\tag{7}$$

Jacobi algorithm:

In [49]:
def jacobi(x0, A, b, kmax, tolerance):
    # initial guess x0 to the solution, diagonal dominant matrix A,
    # right-hand side vector b, convergence criterion tolerance
    bNorm = np.linalg.norm(b)
    n = len(A)
    x = x0
    for k in range(1,kmax):
        for i in range(1,n):
            x[i] = (b[i] - (A[i,:]*x0[:] - A[i,i]*x0[i])) / A[i,i]
        r = b - np.matmul(A,x)
        residual[k] = np.norm(r)
        x0 = x
    return x, residual

#### Gauss-Seidel method

Looking at formula $\eqref{eq:7}$ in Jacobi method we notice that when we are computing $x_{i}^{(k+1)}$, we already have values $x_{1}^{(k+1)},...,x_{i-1}^{(k+1)}$. As we expect for iterations to converge, it is reasonable to assume that values $x_{1}^{(k+1)},...,x_{i-1}^{(k+1)}$ are better than $x_{1}^{(k)},...,x_{i-1}^{(k)}$. For that reason we introduce the following $x^{(k+1)}$ computing formula:

$$ x_i^{(k+1)}=\frac{1}{a_{ii}}(b_{i}-\sum_{j=1}^{i-1}a_{ij}x_{j}^{(k+1)}-\sum_{j=i+1}^{n}a_{ij}x_{j}^{(k)}),\; i=1,...,n\label{eq:8}\tag{8}$$

In terms of $\eqref{eq:5}$, the method looks like:

$$ x^{(k+1)}=Gx^{(k)}+(I-L)^{-1}D^{-1}b,\; G = (I-L)^{-1}U\label{eq:9}\tag{9}$$

Gauss-Seidel algorithm:

In [46]:
def gaussSeidel(x0, A, b, tolerance):
    # initial guess x0 to the solution, diagonal dominant matrix A,
    # right-hand side vector b, convergence criterion tolerance
    bNorm = np.linalg.norm(b)
    n = len(A)
    x = x0
    residual = np.norm(b - np.matmul(A,x))
    while tolerance > residual:
        for i in range(1,n):
            x[i] = (b[i] - A[i,1:i-1]*x[1:i-1] - A[i,i+1:n]*x0[i+1:n]) / A[i,i]
        residual = b - np.matmul(A,x)
        res[k] = np.norm(r)
        x0 = x
    return x, res

#### SOR method

The SOR method is a variant of the Gauss–Seidel method for solving a linear system of equations, resulting in faster convergence. A similar method can be used for any slowly converging iterative process. In practice, in the Gauss-Seidel method we insert one parameter $\omega\in\mathbb{R}$ and then we try and adjust $\omega$ to achieve faster convergence. While doing so, we continue to hold the general idea of Gauss-Seidel method - always use the newest data:

$$ x_i^{(k+1)}=(1-\omega)x_{i}^{(k)} + \frac{\omega}{a_{ii}}(b_{i}-\sum_{j=1}^{i-1}a_{ij}x_{j}^{(k+1)}-\sum_{j=i+1}^{n}a_{ij}x_{j}^{(k)}),\; i=1,...,n\label{eq:10}\tag{10}$$

In terms of $\eqref{eq:5}$, the method looks like:

$$ x^{(k+1)}=Sx^{(k)}+\omega(I-\omega L)^{-1}D^{-1}b,\; S = (I-\omega L)^{-1}((1-\omega)I+\omega U)\label{eq:11}\tag{11}$$

In [51]:
def sor(x0, A, b, tolerance, omega):
    # initial guess x0 to the solution, diagonal dominant matrix A,
    # right-hand side vector b, convergence criterion tolerance
    # parameter omega
    bNorm = np.linalg.norm(b)
    n = len(A)
    x = x0
    residual = np.norm(b - np.matmul(A,x))
    while tolerance > residual:
        for i in range(1,n):
            x[i] = (1 - omega)*x0[i] + omega*(b[i] - A[i,1:i-1]*x[1:i-1] - A[i,i+1:n]*x0[i+1:n]) / A[i,i]
        residual = b - np.matmul(A,x)
        res[k] = np.norm(r)
        x0 = x
    return x, res

Iterative method: https://en.wikipedia.org/wiki/Iterative_method

Jacobi method: https://en.wikipedia.org/wiki/Jacobi_method

Gauss-Seidel method: https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method

SOR method: https://en.wikipedia.org/wiki/Successive_over-relaxation

